In [27]:
import numpy as np
from class_cifar_dataset import CifarDataset
from torchvision import transforms
from torch.utils.data import DataLoader

def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

def reshape(data):
    data = data.reshape(len(data),3, 32, 32)
    data = data.transpose(0, 2, 3, 1)
    return data

train_labels = []
train_data = np.empty((0, 3072), dtype=np.uint8)
test_labels = []
test_data = np.empty((0, 3072), dtype=np.uint8)


for i in range(1, 6):
    file = f'cifar-10-batches-py/data_batch_{i}'
    dictionary = unpickle(file)
    print(dictionary[b'data'].shape)
    train_labels += dictionary[b'labels']
    train_data = np.concatenate([train_data, dictionary[b'data']])
    print("typ pustego")
    print(type(train_data[0][0]))
    # print(type(train_data))
    #print(type( dictionary[b'data']))


file = f'cifar-10-batches-py/test_batch'
dictionary = unpickle(file)
test_labels = dictionary[b'labels']
test_data = dictionary[b'data']

train_data = train_data.reshape(len(train_data), 3, 32, 32)
train_data = train_data.transpose(0, 2, 3, 1)
test_data = test_data.reshape(len(test_data), 3, 32, 32)
test_data = test_data.transpose(0, 2, 3, 1)

print(type(train_data[0][0][0][0]))
print(train_data[0])



for key in dictionary.keys():
    print(f'{key}   {type(dictionary[key])}')

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


trainset = CifarDataset(train_labels, train_data, transform)
train_dataloader = DataLoader(trainset, batch_size=4, shuffle=True)


testset = CifarDataset(test_labels, test_data, transform)
test_dataloader = DataLoader(testset, batch_size=4, shuffle=True)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
# print("typ")

print(trainset.__getitem__(0)[0])

(10000, 3072)
typ pustego
<class 'numpy.uint8'>
(10000, 3072)
typ pustego
<class 'numpy.uint8'>
(10000, 3072)
typ pustego
<class 'numpy.uint8'>
(10000, 3072)
typ pustego
<class 'numpy.uint8'>
(10000, 3072)
typ pustego
<class 'numpy.uint8'>
<class 'numpy.uint8'>
[[[ 59  62  63]
  [ 43  46  45]
  [ 50  48  43]
  ...
  [158 132 108]
  [152 125 102]
  [148 124 103]]

 [[ 16  20  20]
  [  0   0   0]
  [ 18   8   0]
  ...
  [123  88  55]
  [119  83  50]
  [122  87  57]]

 [[ 25  24  21]
  [ 16   7   0]
  [ 49  27   8]
  ...
  [118  84  50]
  [120  84  50]
  [109  73  42]]

 ...

 [[208 170  96]
  [201 153  34]
  [198 161  26]
  ...
  [160 133  70]
  [ 56  31   7]
  [ 53  34  20]]

 [[180 139  96]
  [173 123  42]
  [186 144  30]
  ...
  [184 148  94]
  [ 97  62  34]
  [ 83  53  34]]

 [[177 144 116]
  [168 129  94]
  [179 142  87]
  ...
  [216 184 140]
  [151 118  84]
  [123  92  72]]]
b'batch_label'   <class 'bytes'>
b'labels'   <class 'list'>
b'data'   <class 'numpy.ndarray'>
b'filenames'  

In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [29]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [40]:
import torch
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 0.702
[1,  4000] loss: 0.739
[1,  6000] loss: 0.747
[1,  8000] loss: 0.753
[1, 10000] loss: 0.786
[1, 12000] loss: 0.779
[2,  2000] loss: 0.664
[2,  4000] loss: 0.708
[2,  6000] loss: 0.767
[2,  8000] loss: 0.750
[2, 10000] loss: 0.767
[2, 12000] loss: 0.766
Finished Training


In [41]:
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join(f'{classes[predicted[j]]:5s}'
                              for j in range(4)))

Predicted:  car   frog  dog   car  


In [42]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in test_dataloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 61 %


In [43]:
PATH = './cifar_net2.pth'
torch.save(net.state_dict(), PATH)